In [ ]:
#set env
import os
os.environ["ANTHROPIC_API_KEY"]=""

In [ ]:
import os
import json
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

display(HTML("""
<style>
.config-container {
    border: 1px solid #e0e0e0;
    border-radius: 10px;
    padding: 20px;
    margin: 10px 0;
    background-color: #f8f9fa;
    box-shadow: 0 3px 10px rgba(0,0,0,0.05);
    max-width: 600px;
}
.config-title {
    margin-top: 0;
    margin-bottom: 15px;
    color: #333;
    font-size: 1.5em;
    border-bottom: 1px solid #e0e0e0;
    padding-bottom: 10px;
}
.config-desc {
    color: #555;
    margin-bottom: 20px;
}
.config-success {
    background-color: #d4edda;
    color: #155724;
    padding: 10px;
    border-radius: 5px;
    margin-top: 10px;
    border-left: 4px solid #28a745;
}
.config-warning {
    background-color: #fff3cd;
    color: #856404;
    padding: 10px;
    border-radius: 5px;
    margin-top: 10px;
    border-left: 4px solid #ffc107;
}
/* New styles for improved form field labels */
.widget-label {
    font-weight: bold !important;
    font-size: 1.05em !important;
    color: #333 !important;
    min-width: 160px !important;
    max-width: 160px !important;
}
.widget-inline-hbox {
    width: 100% !important;
    display: flex !important;
    flex-direction: row !important;
    align-items: center !important;
}
.widget-password,
.widget-text {
    width: 100% !important;
}
.field-help-text {
    margin-top: 2px;
    margin-left: 160px;
    font-size: 0.85em;
    color: #666;
    font-style: italic;
}
.field-group {
    margin-bottom: 15px;
    border-bottom: 1px solid #f0f0f0;
    padding-bottom: 10px;
}
.jp-Notebook-cell:first-child .jp-Cell-inputWrapper,
.jp-Notebook-cell:first-child .jp-Cell-outputWrapper {
    display: none !important;
}

body > div:empty, 
.voila-container > div:empty,
.voila-header {
    display: none !important;
}
</style>
"""))

CONFIG_FILE = 'user_config.json'

def load_config():
    """Load configuration from file if it exists"""
    if os.path.exists(CONFIG_FILE):
        try:
            with open(CONFIG_FILE, 'r') as f:
                return json.load(f)
        except:
            return {}
    return {}

def save_config(config):
    """Save configuration to file"""
    with open(CONFIG_FILE, 'w') as f:
        json.dump(config, f)
    
def set_env_from_config(config):
    """Set environment variables from config"""
    if 'ANTHROPIC_API_KEY' in config and config['ANTHROPIC_API_KEY']:
        os.environ["ANTHROPIC_API_KEY"] = config['ANTHROPIC_API_KEY']

config = load_config()

api_key_input = widgets.Password(
    description='Anthropic API Key:',
    placeholder='Enter your Anthropic API key',
    value=config.get('ANTHROPIC_API_KEY', ''),
    layout=widgets.Layout(width='100%'),
    style={'description_width': 'initial'}
)

api_key_help = widgets.HTML(
    value='<div class="field-help-text">Required for Claude AI model access</div>'
)

title_html = widgets.HTML(value='<div class="config-title">🔑 API Configuration</div>')
description_html = widgets.HTML(
    value='<div class="config-desc">Enter your Anthropic API key below to use with the application. Keys are stored locally in a configuration file.</div>'
)

save_button = widgets.Button(
    description='Save Configuration',
    button_style='primary',
    icon='save',
    layout=widgets.Layout(width='auto', margin='20px 0 0 0')
)

status_output = widgets.Output()


def on_save_button_clicked(b):
    with status_output:
        clear_output()
        
        new_config = {
            'ANTHROPIC_API_KEY': api_key_input.value
        }
        save_config(new_config)
        set_env_from_config(new_config)
        
        display(HTML('<div class="config-success">✅ Configuration saved successfully! API key have been set for this session. Reload the page for the changes to go into effect.</div>'))

save_button.on_click(on_save_button_clicked)

form_items = [
    title_html,
    description_html,
    widgets.HTML('<div class="field-group">'),
    api_key_input,
    api_key_help,
    widgets.HTML('</div>'),
    save_button,
    status_output
]

config_container = widgets.VBox(
    form_items,
    layout=widgets.Layout(
        margin='10px 0',
        padding='0',
        width='auto'
    )
)

if not config.get('ANTHROPIC_API_KEY'):
    display(widgets.VBox([config_container]))
    display(HTML('<div class="config-warning">⚠️ Please enter your API key, click \'Save Configuration\', and reload the page!</div>'))
else:
    set_env_from_config(config)
    display(HTML(f'<div class="config-success">✅ Configuration loaded from the config file :)</div>'))
    
    def show_config_form():
        display(widgets.VBox([config_container]))


In [ ]:
# Install required packages only if they're not already installed
import importlib.util
import sys
from IPython.display import display, HTML

required_packages = {
    'langchain_anthropic': 'langchain-anthropic',
    'voila': 'voila',
    'langgraph': 'langgraph',
    'ipywidgets': 'ipywidgets',
    'langchain': 'langchain'
}

def is_package_installed(package_name):
    """Check if a package is installed."""
    return importlib.util.find_spec(package_name) is not None

# Check and install missing packages
missing_packages = []
for import_name, pip_name in required_packages.items():
    if not is_package_installed(import_name):
        missing_packages.append(pip_name)


In [ ]:
from langchain_anthropic import ChatAnthropic
# llm = ChatAnthropic(model="claude-3-5-haiku-latest") # Fastest
llm = ChatAnthropic(model="claude-3-5-sonnet-latest") # More knowledgeable by slower
# llm = ChatAnthropic(model="claude-3-7-sonnet-latest") # Thinking model so high level knowledge but slowest
# all have the same context window of 200k tokens


In [ ]:
from typing import Annotated, Literal

from langchain_core.tools import tool
from langgraph.graph import MessagesState
from langchain_core.messages import RemoveMessage, AIMessage, HumanMessage, ToolMessage

from langgraph.prebuilt import InjectedState
from langchain_core.tools.base import InjectedToolCallId

#TOOL: prompt user for more information
@tool
def prompt_user(prompt: str):
    """Use this tool to prompt the user for additional information
    
    Args:
        prompt: string to prompt user with
    """
    return f"[GUI_PROMPT_NEEDED]{prompt}"

In [ ]:
#json db lookup tool

import json
import re

@tool
def cve_database(query: str, json_file: str = 'nvdcve-1.1-modified.json'):
    """Lookup CVE Database
    
    Args:
        query: keyword query to search database"""
    #print(f"TOOL QUERY: {query}")
    
    try:
        with open(json_file, 'r', encoding='utf-8') as file:
            data = json.load(file)
        
        # Extract CVE_Items
        cve_items = data.get('CVE_Items', [])

        parts = query.split(maxsplit=1)
        name, version = parts[0], parts[1]
        version_parts = version.split('.')
        rng = range(len(version_parts))

        
        subqueries = [name] + [f"{name} {'.'.join(version_parts[:i+1])}" for i in rng]

        
        filtered_results = []

        
        for item in cve_items:
            # Search in multiple fields
            search_context = {
                'cve_id': item['cve']['CVE_data_meta']['ID'],
                'description': item['cve']['description']['description_data'][0]['value'],
                'references': ' '.join([ref['url'] for ref in item['cve']['references']['reference_data']]),
                'assigner': item['cve']['CVE_data_meta']['ASSIGNER']
            }

            for qs in subqueries:
                # Case-insensitive search across multiple fields
                if any(qs.lower() in str(value).lower() for value in search_context.values()):
                    filtered_results.append({
                        'CVE_ID': search_context['cve_id'],
                        'Description': search_context['description'],
                        'References': search_context['references']
                    })
            
        return filtered_results[:5]
    
    except FileNotFoundError:
        return f"Error: File {json_file} not found"
    except json.JSONDecodeError:
        return f"Error: Invalid JSON in {json_file}"
    except Exception as e:
        return f"Unexpected error: {str(e)}"

In [ ]:
import json

@tool
def tooling_database(query: str, json_file: str = 'tooling.json'):
    """Lookup Tooling Database
    
    Args:
        query: keyword query to search database
    """
    #print(f"TOOL QUERY: {query}")
    
    try:
        with open(json_file, 'r', encoding='utf-8') as file:
            data = json.load(file)
        
        # Extract tools from the data
        tools = data.get('tools', [])
        
        # Parse query safely - don't assume structure
        query = query.strip()
        
        # Don't try to split the query if it doesn't have enough parts
        # Just use the whole query as a search term
        search_term = query.lower()
        
        filtered_results = []
        
        for item in tools:
            # Search in multiple fields
            search_context = {
                'title': item.get('tool_title', '').lower(),
                'type': item.get('tool_type', '').lower(),
                'description': item.get('tools_description', '').lower(),
                'tag': item.get('tool_tag', '').lower()
            }
            
            # Add code-related fields if they exist
            if 'standard_code' in item:
                if isinstance(item['standard_code'], dict):
                    search_context['code_type'] = item['standard_code'].get('type', '').lower()
                    search_context['code_example'] = item['standard_code'].get('code', '').lower()
            
            # Check if the search term appears in any field
            if any(search_term in value for value in search_context.values()):
                result = {
                    'Tool': item.get('tool_title', ''),
                    'Type': item.get('tool_type', ''),
                    'Description': item.get('tools_description', '')
                }
                
                # Add optional fields if they exist
                if 'tool_tag' in item:
                    result['Tag'] = item['tool_tag']
                
                if 'standard_code' in item and isinstance(item['standard_code'], dict):
                    result['Code Type'] = item['standard_code'].get('type', '')
                    result['Code Example'] = item['standard_code'].get('code', '')
                
                filtered_results.append(result)
            
        return filtered_results[:5] if filtered_results else "No matching tools found."
    
    except FileNotFoundError:
        return f"Error: File {json_file} not found"
    except json.JSONDecodeError:
        return f"Error: Invalid JSON in {json_file}"
    except Exception as e:
        return f"Unexpected error: {str(e)}"

In [ ]:
TEMPLATE_DIR = "./code_templates"

def load_template(template_name):
    """
    Loads an exploit template from a file.
    
    Parameters:
        template_name (str): The name of the template file (without .py).
    
    Returns:
        str: The content of the template file.
    """
    template_path = os.path.join(TEMPLATE_DIR, f"{template_name}.py")
    
    if os.path.exists(template_path):
        with open(template_path, "r") as file:
            return file.read()
    else:
        return "Template not found."

@tool
def exploit_code(exploit_type: str, params: dict) -> str:
    """
    Loads an exploit template and fills in parameters dynamically.
    
    Parameters:
        exploit_type (str): The name of the exploit template.
        params (dict): A dictionary of parameters to format the template.
    
    Returns:
        str: The formatted exploit code.
    """
    template_code = load_template(exploit_type)
    print (template_code)
    
    if "Template not found" in template_code:
        return "Error: The requested exploit template does not exist."
    
    try:
        formatted_code = template_code.format(**params)
        return formatted_code
    except KeyError as e:
        return f"Error: Missing required parameter {e} in template."

In [ ]:
import json
from langchain_core.tools import tool

@tool
def exploitation_info(category: str = None, json_file: str = 'exploit_db.json'):
    """
    Retrieve information from the exploitation phase of the cyber kill chain.

    Args:
        category (str): One of 'techniques', 'vulnerabilities', 'payloads', 'mitigation_strategies', or 'threat_actors'.
        json_file (str): Path to the JSON file.

    Returns:
        dict or list: Relevant information from the database.
    """
    try:
        with open(json_file, "r") as file:
            exploit_db = json.load(file)

        exploitation_phase = exploit_db.get("exploitation_phase", {})

        if not category:
            return exploitation_phase

        if category in exploitation_phase:
            return exploitation_phase[category]

        return {"error": "Invalid category specified."}

    except FileNotFoundError:
        return f"Error: File {json_file} not found"
    except json.JSONDecodeError:
        return f"Error: Invalid JSON in {json_file}"
    except Exception as e:
        return f"Unexpected error: {str(e)}"

In [ ]:
import json

@tool
def pivot_database(query: str, json_file: str = 'pivot_db.json'):
    """Lookup Pivoting Tooling Database
    
    Args:
        query: keyword query to search database (tool name, type, tag, etc.)
        json_file: path to the JSON file containing the pivoting tools data
    """
    #print(f"TOOL QUERY: {query}")
    
    try:
        with open(json_file, 'r', encoding='utf-8') as file:
            data = json.load(file)
        
        # Extract tools from the data
        tools = data.get('tools', [])
        
        # Parse query safely
        search_term = query.strip().lower()
        
        filtered_results = []
        
        for item in tools:
            # Search in multiple fields
            search_context = {
                'title': item.get('tool_title', '').lower(),
                'type': item.get('tool_type', '').lower(),
                'description': item.get('tools_description', '').lower(),
                'tag': item.get('tool_tag', '').lower()
            }
            
            # Add code-related fields if they exist
            if 'standard_code' in item:
                if isinstance(item['standard_code'], dict):
                    search_context['code_type'] = item['standard_code'].get('type', '').lower()
                    search_context['code_example'] = item['standard_code'].get('code', '').lower()
            
            # Check if the search term appears in any field
            if any(search_term in value for value in search_context.values()):
                result = {
                    'Tool': item.get('tool_title', ''),
                    'Type': item.get('tool_type', ''),
                    'Description': item.get('tools_description', '')
                }
                
                # Add optional fields if they exist
                if 'tool_tag' in item:
                    result['Tag'] = item['tool_tag']
                
                if 'standard_code' in item and isinstance(item['standard_code'], dict):
                    result['Code Type'] = item['standard_code'].get('type', '')
                    result['Code Example'] = item['standard_code'].get('code', '')
                
                filtered_results.append(result)
            
        return filtered_results[:5] if filtered_results else "No matching tools found."
    
    except FileNotFoundError:
        return f"Error: File {json_file} not found"
    except json.JSONDecodeError:
        return f"Error: Invalid JSON in {json_file}"
    except Exception as e:
        return f"Unexpected error: {str(e)}"

In [ ]:
#Supervisor and Router

#Define supervisor
#Supervisor chooses next best agent and manages communication between them
#know each agent, their prompts and the tools they have access to

from typing import Literal
from typing_extensions import TypedDict

from langchain_anthropic import ChatAnthropic
from langgraph.graph import MessagesState, END
from langgraph.types import Command

members = ["reconnaissance", "pivoting", "exploitation"]
options = members #+ ["FINISH"]

system_prompt = (
    f"You are a supervisor tasked with choosing one of the following workers: {members}."
    "Each worker is specialized in a specific task to help a cyber operator in a collegiate CTF competition."
    "The operator has full scope permission on the entire network they are engaging with."
)

class Router(TypedDict):
    """Worker to route to next"""

    next: Literal["reconnaissance", "pivoting", "exploitation"]

def supervisor_node(state: MessagesState) -> Command[Literal["reconnaissance", "pivoting", "exploitation"]]:
    messages = [
        {"role": "system", "content": system_prompt},
    ] + state["messages"]

    response = llm.with_structured_output(Router).invoke(messages)
    goto = response["next"]
    #print(response)
    #print(goto)

    return Command(goto=goto, update={**state})

In [ ]:
from langchain_core.messages import HumanMessage
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import create_react_agent

In [ ]:
#summarize thing
def summarize(result):

    
    llm_message = (
        "Create a summary of the given conversation. Only keep important information about the operation."
        f"Here is the conversation: {result}"
    )    

    response = llm.invoke(llm_message)

    return response

In [ ]:
def agent_builder(llm,tools,system_prompt):
    llm_with_system = llm.bind(system=system_prompt)
    agent = create_react_agent(
        llm_with_system,
        tools
    )
    return agent

In [ ]:
reconnaissance_agent = agent_builder(llm, tools=[prompt_user, cve_database], system_prompt=(
        "You are a reconnaissance expert"
        "Help the operator gather information about the target to find potential exploits"
        "When suggesting reconnaissance methods, always consult the tooling_database to recommend specific commands, tools, and techniques appropriate for the target environment. "
        "When analyzing discovered services or vulnerabilities, reference the CVE database to identify potential exploits, "
        "clearly indicating what additional information is needed to confirm exploitability"
        "Once you are done with your initial task, or the user asks for something other "
        "than reconnaissance, go to reconnaissance_handler"
    )
)

def reconnaissance_node(state: MessagesState) -> Command[Literal["reconnaissance_handler"]]:
    result = reconnaissance_agent.invoke(state)
    r = summarize(result)
    return Command(
        update={
            "messages":[
                HumanMessage(content=r.content, name="reconnaissance")
            ]
        },
        goto="reconnaissance_handler"
    )

def reconnaissance_handler_node(state: MessagesState) -> Command[Literal["reconnaissance", "exploitation", "__end__"]]:
    try:
        while True:
            nextCase = input(
                "Reconnaissance Complete ᕕ(⌐■_■)ᕗ ♪♬\n\n"
                "1) Stay Here\n"
                "2) Go to exploitation\n"
                "3) Make it end\n"
                ">>> "
            ).strip()

            if nextCase == '1':
                nextState = "reconnaissance"
                update = input("\n\nAnything specific?\n\n>>> ").strip()
                break
            elif nextCase == '2':
                nextState = "exploitation"
                update = input("\n\nAnything specific?\n\n>>> ").strip()
                break
            elif nextCase == '3':
                nextState = "__end__"
                update = ""
                break
            else:
                print("\nInvalid input. Please enter '1', '2', or '3'.\n")
    except EOFError:
        print("\nEOF detected. Ending session.")
        nextState = "__end__"
        update = ""

    return Command(
        update={
            "messages": [
                HumanMessage(content=update, name="user")
            ]
        },
        goto=nextState
    )

In [ ]:
exploitation_agent = agent_builder(llm, tools=[prompt_user, cve_database], system_prompt=(
        # "you are an exploitation specialist for a CTF cyber operation."
        # "Give specific information and steps in order to exploit a given potential vulnerability."

        
        "You are an exploitation specialist for a CTF cyber operation. "
        "When approached with a vulnerability or target, FIRST query the cve_database tool using relevant keywords. "
        "Return only the most relevant exploits from the database that match the query. "
        "If more information is needed to find appropriate exploits, ask specific follow-up questions to narrow down the search. "
        "Provide specific information and steps in order to exploit the given potential vulnerability based on the CVE database results."
    )
)

def exploitation_node(state: MessagesState) -> Command[Literal["exploitation_handler"]]:
    result = exploitation_agent.invoke(state)
    r = summarize(result)
    return Command(
        update={
            "messages":[
                HumanMessage(content=r.content, name="exploitation")
            ]
        },
        goto="exploitation_handler",
    )

def exploitation_handler_node(state: MessagesState) -> Command[Literal["reconnaissance", "exploitation", "__end__"]]:
    try:
        while True:
            nextCase = input(
                "Exploitation Complete ᕕ(⌐■_■)ᕗ ♪♬\n\n"
                "1) Stay Here\n"
                "2) Go to pivoting\n"
                "3) Make it end\n"
                ">>> "
            ).strip()

            if nextCase == '1':
                nextState = "exploitation"
                update = input("\n\nAnything specific?\n\n>>> ").strip()
                break
            elif nextCase == '2':
                nextState = "pivoting"
                update = input("\n\nAnything specific?\n\n>>> ").strip()
                break
            elif nextCase == '3':
                nextState = "__end__"
                update = ""
                break
            else:
                print("\nInvalid input. Please enter '1', '2', or '3'.\n")
    except EOFError:
        print("\nEOF detected. Ending session.")
        nextState = "__end__"
        update = ""

    return Command(
        update={
            "messages": [
                HumanMessage(content=update, name="user")
            ]
        },
        goto=nextState
    )


In [ ]:
pivoting_agent = agent_builder(llm, tools=[prompt_user, cve_database], system_prompt=(
         "You are a network pivoting specialist for a CTF cyber operation. "
        "When approached with a request, FIRST query the pivot_database tool using relevant keywords. "
        "If tools are found in the database, prioritize these recommendations and include their exact "
        "descriptions and command examples as provided in the database. "
        "If the pivot_database returns no results or incomplete information, clearly inform the user "
        "with a statement like 'NOTE: This technique/tool is not in the authorized database.' Then, "
        "you may suggest alternative approaches that might be effective, but clearly mark these as "
        "supplementary information not from the database. "
        "Structure your responses with clear distinctions between database-verified techniques and "
        "supplementary suggestions. Always provide step-by-step instructions when possible, and "
        "favor practical, actionable advice over theoretical explanations."
    )
)

def pivoting_node(state: MessagesState) -> Command[Literal["exploitation_handler"]]:
    result = exploitation_agent.invoke(state)
    r = summarize(result)
    return Command(
        update={
            "messages":[
                HumanMessage(content=r.content, name="exploitation")
            ]
        },
        goto="exploitation_handler",
    )

def pivoting_handler_node(state: MessagesState) -> Command[Literal["pivoting", "__end__"]]:
    try:
        while True:
            nextCase = input(
                "Pivoting Complete ᕕ(⌐■_■)ᕗ ♪♬\n\n"
                "1) Stay Here\n"
                "2) Make it end\n\n"
                ">>> "
            ).strip()
            
            if nextCase == '1':
                nextState = "pivoting"
                update = input("\n\nAnything specific?\n\n>>> ").strip()
                break
            elif nextCase == '2':
                nextState = "__end__"
                update = ""
                break
            else:
                print("\nInvalid input. Please enter '1' or '2'.\n")
    except EOFError:
        print("\nEOF detected. Ending session.")
        nextState = "__end__"
        update = ""

    return Command(
        update={
            "messages": [
                HumanMessage(content=update, name="user")
            ]
        },
        goto=nextState
    )

In [ ]:
pivoting_agent = agent_builder(llm, tools=[prompt_user, cve_database], system_prompt=(
        "you are an pivoting specialist for a CTF cyber operation."
        "Give specific information and steps in order to pivot and move laterally in a network."
    )
)

def pivoting_node(state: MessagesState) -> Command[Literal["pivoting_handler"]]:
    result = pivoting_agent.invoke(state)
    r = summarize(result)
    return Command(
        update={
            "messages":[
                HumanMessage(content=r.content, name="pivoting")
            ]
        },
        goto="pivoting_handler"
    )

def pivoting_handler_node(state: MessagesState) -> Command[Literal["pivoting", "__end__"]]:
    nextCase = input(
        "Pivoting Complete ᕕ(⌐■_■)ᕗ ♪♬\n\n"
        "1) Stay Here\n"
        "2) Make it end\n\n"
        ">>> "
    )
    match nextCase:
        case '1':
            nextState="pivoting"
            update = input("\n\nanything specific?\n\n>>>")
        case '2':
            nextState=END
            update = ""

    return Command(
        update={
            "messages":[
                HumanMessage(content=update, name="user")
            ]
        },
        goto=nextState
    )

In [ ]:
builder = StateGraph(MessagesState)

builder.add_edge(START, "supervisor")
builder.add_node("supervisor", supervisor_node)

builder.add_node("reconnaissance", reconnaissance_node)
builder.add_node("reconnaissance_handler", reconnaissance_handler_node)

builder.add_node("exploitation", exploitation_node)
builder.add_node("exploitation_handler", exploitation_handler_node)

builder.add_node("pivoting", pivoting_node)
builder.add_node("pivoting_handler", pivoting_handler_node)

graph = builder.compile()

In [ ]:
from IPython.display import Image, display
#display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
import pprint
# pprint.pprint(graph.get_graph().to_json())

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from langchain.schema import AIMessage, HumanMessage
import re

display(HTML('''
<style>
.widget-hbox.widget-input-area {
    display: flex !important;
    flex-direction: row !important;
    justify-content: space-between !important;
    width: 100% !important;
}

.widget-hbox > .widget-text {
    flex: 0 0 85% !important;
    width: 85% !important;
    max-width: 85% !important;
}

.widget-hbox > .widget-button {
    flex: 0 0 14% !important;
    width: 14% !important;
    max-width: 14% !important;
}

.widget-text input {
    width: 100% !important;
    box-sizing: border-box !important;
    height: 40px !important;
    padding: 8px 12px !important;
    border: 2px solid #c2c8d1 !important;
    border-radius: 8px !important;
    font-size: 15px !important;
    background-color: white !important;
}

.jupyter-button {
    height: 40px !important;
    font-size: 14px !important;
    width: 100% !important;
}

.clear-chat-button {
    margin-top: 10px !important;
    width: 100% !important;
}


</style>
'''))

display(HTML('<div class="chat-main-container">'))

title = widgets.HTML(value='<h2 class="app-title" style="text-align: center; margin: 10px 0 20px; font-weight: 600; color: #333;">Maestro</h2>')
chat_area = widgets.Output(layout={'width': '100%', 'height': '450px', 'overflow': 'auto', 'border': 'none'})

input_html = '''
<div style="display: flex; width: 100%; margin: 10px 0; gap: 1%;">
    <input type="text" id="user_input" placeholder="Ask your cyber security question..." 
           style="flex: 0 0 85%; width: 85%; height: 40px; padding: 8px 12px; 
                  border: 2px solid #c2c8d1; border-radius: 8px; font-size: 15px;">
    <button id="send_button" 
            style="flex: 0 0 14%; width: 14%; height: 40px; background-color: #4361ee; 
                   color: white; border: none; border-radius: 8px; cursor: pointer; font-size: 14px;">
        Send
    </button>
</div>
'''

user_input = widgets.Text(
    description='', 
    placeholder='Ask your cyber security question...',
    layout=widgets.Layout(width='100%'), 
    style={'description_width': 'auto'}
)
send_button = widgets.Button(
    description='Send',
    button_style='primary',
    icon='paper-plane',
    layout=widgets.Layout(width='100%')
)
input_area = widgets.HBox([user_input, send_button], layout=widgets.Layout(width='100%'))

clear_button = widgets.Button(
    description='Clear Chat',
    button_style='danger',
    icon='trash',
    layout=widgets.Layout(width='100%', class_='clear-chat-button')
)

status_indicator = widgets.HTML(value="")

user_style = """
<div class="chat-fade-in user-message" style="padding: 14px 18px; border-radius: 18px 18px 4px 18px; margin: 12px 0px 12px 20%; width: 80%; box-shadow: 0 3px 10px rgba(0,0,0,0.04); background-color: #e6f2ff;">
    <span style="font-weight: 600; color: #4361ee; font-size: 0.9rem;">You</span>
    <div style="margin-top: 8px; line-height: 1.5;">{}</div>
</div>
"""

assistant_style = """
<div class="chat-fade-in assistant-message" style="padding: 14px 18px; border-radius: 18px 18px 18px 4px; margin: 12px 20% 12px 0px; width: 80%; box-shadow: 0 3px 10px rgba(0,0,0,0.04); background-color: white;">
    <span style="font-weight: 600; color: #6c757d; font-size: 0.9rem;">Assistant</span>
    <div style="margin-top: 8px; white-space: pre-wrap; line-height: 1.5;">{}</div>
</div>
"""

system_style = """
<div class="chat-fade-in" style="color: #6c757d; font-style: italic; text-align: center; margin: 15px 0; font-size: 0.95rem; background-color: #f8f9fa; padding: 12px; border-radius: 12px; border: 1px dashed #e0e0e0;">{}</div>
"""

status_style = """
<div id="status-indicator" class="status-container" style="background-color: #f8f9fa; border-left: 4px solid {}; padding: 12px 18px; margin: 15px 0; text-align: left; border-radius: 12px; box-shadow: 0 3px 12px rgba(0,0,0,0.04); display: flex; align-items: center;">
    <span style="font-weight: 600; color: {}; margin-right: 10px; font-size: 0.95rem;">{}</span>
    <span style="color: #555;">{}</span>
</div>
"""

def update_status(status_type, message):
    """
    Update the status indicator with different styles based on status type
    """
    colors = {
        "info": ("#3498db", "#3498db"), 
        "processing": ("#f39c12", "#f39c12"),
        "success": ("#2ecc71", "#2ecc71"),
        "error": ("#e74c3c", "#e74c3c"),
        "waiting": ("#9b59b6", "#9b59b6")
    }
    
    border_color, text_color = colors.get(status_type, ("#888", "#888"))
    status_title = status_type.capitalize()
    
    icons = {
        "info": "ℹ️",
        "processing": "⚙️",
        "success": "✅",
        "error": "❌",
        "waiting": "⏳"
    }
    
    icon = icons.get(status_type, "")
    
    status_indicator.value = status_style.format(
        border_color, text_color, f"{icon} {status_title}", message
    )

def on_send_button_clicked(b):
    question = user_input.value
    if not question.strip():
        return
    
    with chat_area:
        display(HTML(user_style.format(question)))
    
    user_input.value = ''
    
    user_input.placeholder = 'Ask your cyber security question...'
    
    update_status("processing", "Analyzing your request and routing to appropriate agent...")
    
    try:
        events = graph.stream(
            {
                "messages": [
                    (
                        "user",
                        f"The operator's question: {question}"
                    )
                ],
            },
            subgraphs=True,
        )
        
        for s in events:
            agent = s[1]
            x, y = next(iter(agent.items()))
            key, message = next(iter(y.items()))
            node_message = message[0]
            
            agent_name = x if x in ["reconnaissance", "exploitation", "pivoting"] else "assistant"
            update_status("info", f"Response from {agent_name.capitalize()}")
            
            with chat_area:
                if isinstance(node_message, AIMessage):
                    if isinstance(node_message.content, str) and "[GUI_PROMPT_NEEDED]" in node_message.content:
                        prompt_text = node_message.content.split("[GUI_PROMPT_NEEDED]")[1]
                        display(HTML(assistant_style.format(f"I need more information: {prompt_text}")))
                        
                        update_status("waiting", f"Waiting for your response")
                        short_prompt = prompt_text[:30] + "..." if len(prompt_text) > 30 else prompt_text
                        user_input.placeholder = f"Reply: {short_prompt}"
                    else:
                        if isinstance(node_message.content, list) and len(node_message.content) > 0 and isinstance(node_message.content[0], dict):
                            message_text = node_message.content[0].get("text", str(node_message.content))
                        else:
                            message_text = str(node_message.content)
                        
                        formatted_text = message_text.replace('\n', '<br>')
                        bullet_pattern = r'(^|\<br\>)(\*|\-|\•)\s'
                        formatted_text = re.sub(bullet_pattern, r'\1• ', formatted_text)
                        
                        display(HTML(assistant_style.format(formatted_text)))
                        
                        update_status("success", "Response complete")
                elif isinstance(node_message, ToolMessage) and "[GUI_PROMPT_NEEDED]" in node_message.content:
                    prompt_text = node_message.content.split("[GUI_PROMPT_NEEDED]")[1]
                    display(HTML(assistant_style.format(f"I need more information: {prompt_text}")))
                    
                    short_prompt = prompt_text[:30] + "..." if len(prompt_text) > 30 else prompt_text
                    user_input.placeholder = f"Reply: {short_prompt}"
                    
                    update_status("waiting", f"Waiting for your response")
                else:
                    display(HTML(assistant_style.format(str(node_message))))
                    update_status("success", "Response complete")
    
    except Exception as e:
        with chat_area:
            display(HTML(system_style.format(f"Error: {str(e)}")))
        update_status("error", f"An error occurred: {str(e)}")

def on_clear_button_clicked(b):
    with chat_area:
        clear_output()
    status_indicator.value = ""
    user_input.placeholder = 'Ask your cyber security question...'
    initialize_chat()

send_button.on_click(on_send_button_clicked)
clear_button.on_click(on_clear_button_clicked)

def on_value_change(change):
    if change['type'] == 'change' and change['name'] == 'value' and user_input.value.strip():
        on_send_button_clicked(None)

user_input.continuous_update = False
user_input.observe(on_value_change, 'value')

chat_widget = widgets.VBox([
    title, 
    chat_area, 
    status_indicator, 
    input_area, 
    widgets.HBox([clear_button], layout=widgets.Layout(justify_content='flex-end', width='100%', margin='5px 0 15px'))
], layout=widgets.Layout(margin='0', width='100%', max_width='100%'))

display(chat_widget)
display(HTML('</div>'))

def initialize_chat():
    with chat_area:
        display(HTML(system_style.format("Welcome to Maestro. Ask me a Cyber Attack Question. For example: Guide me through enumeration of my target: hackme.htb (10.10.5.7)")))
    update_status("info", "Ready for your questions")

initialize_chat()